# Supervised Learning: Feature Engineering & Data Preparation
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1kXhe1hFcZFeA-THfQ1m8KzQQR97vDbeL?usp=sharing)

## Objective
Prepare datasets for multiple supervised learning models to predict student success:
1. **Early Warning System** - Predict final grade bin from first semester/year
2. **Semester-by-Semester Probation Prediction** - Predict if student will be on probation at semester end
3. **Academic Recovery** - Predict if student will recover after first probation
4. **Next Semester GPA Change** - Predict GPA change in next semester (regression)
5. **STEM Course Success** - Predict if student will pass a STEM course


## Success Definitions
- **Dean's List**: CGPA ≥ 3.5
- **Good Standing**: 2.5 ≤ CGPA < 3.5
- **At-Risk**: 2.0 ≤ CGPA < 2.5
- **Probation**: CGPA < 2.0 (determined at END of each semester)


Note
- **Probation**: Semester-by-semester prediction tracking ALL instances (students can recover and fall back)
- **Academic Recovery**: First probation → next semester recovery (critical for dismissal prevention)

In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings

warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully")

Libraries imported successfully


## 1. Load Data

In [69]:
cgpa_df = pd.read_csv('anon_CGPA_Reports_v2b.csv')
transcript_df = pd.read_csv('anon_Transcript_Reports_v2.csv')
prereq_df = pd.read_csv('course_prerequisites.csv')

print(f"CGPA Data: {cgpa_df.shape}")
print(f"Transcript Data: {transcript_df.shape}")
print(f"\nUnique students:")
print(f"  CGPA: {cgpa_df['Student Ref'].nunique()}")
print(f"  Transcript: {transcript_df['StudentRef'].nunique()}")
print(f"Course prerequisites loaded: {len(prereq_df)} rows")
print(f"Unique courses with prerequisites: {prereq_df['Course Code'].nunique()}")

print("\nCGPA columns:", cgpa_df.columns.tolist())
print("\nTranscript columns:", transcript_df.columns.tolist())

CGPA Data: (24648, 12)
Transcript Data: (115421, 20)

Unique students:
  CGPA: 3718
  Transcript: 3684
Course prerequisites loaded: 592 rows
Unique courses with prerequisites: 187

CGPA columns: ['Yeargroup', 'Student Ref', 'Admission Year', 'Program', 'Semester/Year', 'Academic Year', 'Student Status', 'Gender', 'Nationality', 'Application Category', 'GPA', 'CGPA']

Transcript columns: ['StudentRef', 'Admission Year', 'Program ID', 'Program', 'Semester/Year', 'Academic Year', 'Student Status', 'Gender', 'Course Code', 'Course Name', 'Mark', 'Grade', 'Grade point', 'Grade system', 'GPA', 'CGPA', 'Subject Credit', 'Calculate toward Graduation Criteria?', 'Course offering plan name', 'Student Type']


## 2. Process CGPA Data (Semester-Level)

In [70]:
cgpa_copy = cgpa_df.copy()

cgpa_copy['GPA'] = pd.to_numeric(cgpa_copy['GPA'], errors='coerce')
cgpa_copy['CGPA'] = pd.to_numeric(cgpa_copy['CGPA'], errors='coerce')

cgpa_copy['Admission_Year'] = cgpa_copy['Admission Year'].str.split('-').str[0].astype(int)

semester_map = {
    'Semester 1': 1,
    'Semester 2': 2,
    'Semester 3': 3
}

def parse_academic_year(year_str):
    try:
        return int(str(year_str).split('-')[0])
    except:
        return np.nan

cgpa_copy['Academic_Year_Numeric'] = cgpa_copy['Academic Year'].apply(parse_academic_year)
cgpa_copy['Semester_Numeric'] = cgpa_copy['Semester/Year'].map(semester_map)

cgpa_copy = cgpa_copy.sort_values(['Student Ref', 'Academic_Year_Numeric', 'Semester_Numeric'])

cgpa_copy['semester_number'] = cgpa_copy.groupby('Student Ref').cumcount() + 1

print("CGPA data processed")
print(f"Total semester records: {len(cgpa_copy)}")
cgpa_copy.head(10)

CGPA data processed
Total semester records: 24648


,Yeargroup,Student Ref,Admission Year,Program,Semester/Year,Academic Year,Student Status,Gender,Nationality,Application Category,GPA,CGPA,Admission_Year,Academic_Year_Numeric,Semester_Numeric,semester_number
16297,2026,S00039f6fd1b74390,2022-2023,B.Sc - Mechanical Engineering,Semester 2,2022-2023,Active,Male,Country0,AFA,3.88,3.88,2022,2022,2,1
20696,2026,S00039f6fd1b74390,2022-2023,B.Sc - Mechanical Engineering,Semester 1,2023-2024,Active,Male,Country0,AFA,3.78,3.82,2022,2023,1,2
20697,2026,S00039f6fd1b74390,2022-2023,B.Sc - Mechanical Engineering,Semester 2,2023-2024,Active,Male,Country0,AFA,3.83,3.83,2022,2023,2,3
20698,2026,S00039f6fd1b74390,2022-2023,B.Sc - Mechanical Engineering,Semester 3,2023-2024,Active,Male,Country0,AFA,3.65,3.78,2022,2023,3,4
22323,2026,S00039f6fd1b74390,2022-2023,B.Sc - Mechanical Engineering,Semester 1,2024-2025,Active,Male,Country0,AFA,3.78,3.78,2022,2024,1,5
23678,2026,S00039f6fd1b74390,2022-2023,B.Sc - Mechanical Engineering,Semester 2,2024-2025,Active,Male,Country0,AFA,3.90,3.80,2022,2024,2,6
222,2016,S000901505ca1ec7f,2012-2013,B.Sc - Business Administration,Semester 1,2015-2016,Active,Male,Country0,Unknown,3.75,3.74,2012,2015,1,1
223,2016,S000901505ca1ec7f,2012-2013,B.Sc - Business Administration,Semester 2,2015-2016,Active,Male,Country0,Unknown,3.63,3.72,2012,2015,2,2
12650,2025,S0021eb5e8ac9bfec,2021-2022,B.Sc - Management Information Systems,Semester 2,2021-2022,Graduated,Female,Country0,AFA,3.75,3.75,2021,2021,2,1
15278,2025,S0021eb5e8ac9bfec,2021-2022,B.Sc - Management Information Systems,Semester 1,2022-2023,Graduated,Female,Country0,AFA,3.17,3.44,2021,2022,1,2


## 3. Create Semester-by-Semester GPA Features

In [71]:
semester_gpa_pivot = cgpa_copy.pivot_table(
    index='Student Ref',
    columns='semester_number',
    values='GPA',
    aggfunc='first'
)

semester_gpa_pivot.columns = [f'semester_{int(col)}_gpa' for col in semester_gpa_pivot.columns]

semester_cgpa_pivot = cgpa_copy.pivot_table(
    index='Student Ref',
    columns='semester_number',
    values='CGPA',
    aggfunc='first'
)

semester_cgpa_pivot.columns = [f'semester_{int(col)}_cgpa' for col in semester_cgpa_pivot.columns]

semester_features = pd.concat([semester_gpa_pivot, semester_cgpa_pivot], axis=1)

print(f"Semester pivot created: {semester_features.shape}")
print(f"Max semesters tracked: {cgpa_copy['semester_number'].max()}")
semester_features.head()

Semester pivot created: (3718, 38)
Max semesters tracked: 19


,semester_1_gpa,semester_2_gpa,semester_3_gpa,semester_4_gpa,semester_5_gpa,semester_6_gpa,semester_7_gpa,semester_8_gpa,semester_9_gpa,semester_10_gpa,...,semester_10_cgpa,semester_11_cgpa,semester_12_cgpa,semester_13_cgpa,semester_14_cgpa,semester_15_cgpa,semester_16_cgpa,semester_17_cgpa,semester_18_cgpa,semester_19_cgpa
Student Ref,,,,,,,,,,,,,,,,,,,,,
S00039f6fd1b74390,3.88,3.78,3.83,3.65,3.78,3.90,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
S000901505ca1ec7f,3.75,3.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
S0021eb5e8ac9bfec,3.75,3.17,2.83,3.25,3.11,3.25,3.13,3.63,3.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
S0027200343737e85,3.88,3.22,3.72,3.61,3.63,3.30,3.11,3.67,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
S002e2924edb73507,3.39,2.78,3.50,3.40,3.30,3.67,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
student_level_cgpa = cgpa_copy.groupby('Student Ref').agg({
    'CGPA': 'last',
    'GPA': ['mean', 'std', 'min', 'max'],
    'Program': 'last',
    'Gender': 'first',
    'Nationality': 'first',
    'Application Category': 'first',
    'Student Status': 'last',
    'Admission_Year': 'first',
    'Yeargroup': 'last',
    'semester_number': 'max'
}).reset_index()

student_level_cgpa.columns = ['Student_Ref', 'Final_CGPA', 'Mean_GPA', 'GPA_Std', 'Min_GPA', 'Max_GPA',
                              'Program', 'Gender', 'Nationality', 'Application_Category',
                              'Student_Status', 'Admission_Year', 'Grad_Year', 'Num_Semesters']

print(f"Student-level CGPA data: {student_level_cgpa.shape}")
student_level_cgpa.head()

Student-level CGPA data: (3718, 14)


,Student_Ref,Final_CGPA,Mean_GPA,GPA_Std,Min_GPA,Max_GPA,Program,Gender,Nationality,Application_Category,Student_Status,Admission_Year,Grad_Year,Num_Semesters
0,S00039f6fd1b74390,3.80,3.803333,0.090037,3.65,3.90,B.Sc - Mechanical Engineering,Male,Country0,AFA,Active,2022,2026,6
1,S000901505ca1ec7f,3.72,3.690000,0.084853,3.63,3.75,B.Sc - Business Administration,Male,Country0,Unknown,Active,2012,2016,2
2,S0021eb5e8ac9bfec,3.35,3.291111,0.286899,2.83,3.75,B.Sc - Management Information Systems,Female,Country0,AFA,Graduated,2021,2025,9
3,S0027200343737e85,3.52,3.126667,1.199833,0.00,3.88,B.Sc - Computer Science,Male,Country6,MCF,Graduated,2017,2021,9
4,S002e2924edb73507,3.21,3.340000,0.301927,2.78,3.67,B.Sc - Business Administration,Male,Country0,Unknown,Graduated,2014,2018,6


## 4. Process Transcript Data (Course-Level)

In [73]:
# Helper function to parse prerequisite course codes from text
def parse_prerequisite_courses(prereq_text):
    """
    Extract course codes from prerequisite text.
    
    Args:
        prereq_text: Text like "CS221 - Discrete Structures, CS222 - Data Structures"
    
    Returns:
        List of normalized course codes: ['CS221', 'CS222']
    """
    if pd.isna(prereq_text) or prereq_text == '-' or prereq_text.strip() == '':
        return []
    
    # Split by comma
    courses = prereq_text.split(',')
    
    # Extract course codes (format: "CODE - Name" or "CODE")
    course_codes = []
    for course in courses:
        course = course.strip()
        if not course or course == '-':
            continue
        
        # Extract the course code (before " - " or before space)
        if ' - ' in course:
            code = course.split(' - ')[0].strip()
        else:
            # Take first word
            code = course.split()[0].strip()
        
        # Normalize: remove spaces
        code = code.replace(' ', '')
        
        # Remove grade requirements like "C or better"
        if code and not any(word in code.lower() for word in ['or', 'better', 'grade']):
            course_codes.append(code.upper())
    
    return course_codes

In [74]:
# Parse prerequisites for all courses
prereq_df['AND_prerequisites'] = prereq_df['Prerequisite-AND'].apply(parse_prerequisite_courses)
prereq_df['OR_prerequisites'] = prereq_df['Prerequisite-OR'].apply(parse_prerequisite_courses)
prereq_df['Course_Code_Normalized'] = prereq_df['Course Code'].str.replace(' ', '').str.upper()

print(f"Courses with AND prerequisites: {(prereq_df['AND_prerequisites'].str.len() > 0).sum()}")
print(f"Courses with OR prerequisites: {(prereq_df['OR_prerequisites'].str.len() > 0).sum()}")
print(f"\nSample parsed prerequisites:")
print(prereq_df[['Course Code', 'AND_prerequisites', 'OR_prerequisites']].head())

Courses with AND prerequisites: 475
Courses with OR prerequisites: 268

Sample parsed prerequisites:
  Course Code AND_prerequisites OR_prerequisites
0       EE341    [SC113, ME101]               []
1       EE341    [SC113, ME101]               []
2       EE341    [SC113, ME101]               []
3      EE 422           [EE321]               []
4     ENGR411         [ENGR312]               []


In [75]:
# Define prerequisite feature calculation function
def calculate_prerequisite_features(student_id, course_code, program, enrollment_semester, 
                                     transcript_df, prereq_lookup):
    """
    Calculate prerequisite-based features for a course enrollment.
    
    Returns:
        dict with keys:
        - all_prerequisites_met: Binary (1/0)
        - avg_grade_in_prerequisites: Float (0-4)
        - min_grade_in_prerequisites: Float (0-4)
        - num_prerequisites_with_low_grades: Integer
    """
    # Get prerequisites for this course in this program
    course_prereqs = prereq_lookup[
        (prereq_lookup['Course_Code_Normalized'] == course_code) &
        (prereq_lookup['Program Name'] == program)
    ]
    
    # Default values if no prerequisite info
    if len(course_prereqs) == 0:
        return {
            'all_prerequisites_met': 1,
            'avg_grade_in_prerequisites': np.nan,
            'min_grade_in_prerequisites': np.nan,
            'num_prerequisites_with_low_grades': 0
        }
    
    prereq_row = course_prereqs.iloc[0]
    and_prereqs = prereq_row['AND_prerequisites']
    or_prereqs = prereq_row['OR_prerequisites']
    
    # Get student's prior transcript (BEFORE this enrollment)
    prior_transcript = transcript_df[
        (transcript_df['StudentRef'] == student_id) &
        (transcript_df['semester_number'] < enrollment_semester)
    ]
    prior_courses = prior_transcript['Course_Code_Normalized'].tolist()
    
    # Check AND prerequisites (all must be completed)
    and_satisfied = all(prereq in prior_courses for prereq in and_prereqs) if len(and_prereqs) > 0 else True
    
    # Check OR prerequisites (at least one must be completed)
    or_satisfied = any(prereq in prior_courses for prereq in or_prereqs) if len(or_prereqs) > 0 else True
    
    all_prerequisites_met = int(and_satisfied and or_satisfied)
    
    # Calculate grade-based features
    all_prereq_courses = and_prereqs + or_prereqs
    if len(all_prereq_courses) > 0:
        completed_prereqs = prior_transcript[prior_transcript['Course_Code_Normalized'].isin(all_prereq_courses)]
        
        if len(completed_prereqs) > 0:
            avg_grade = completed_prereqs['Grade point'].mean()
            min_grade = completed_prereqs['Grade point'].min()
            num_low_grades = (completed_prereqs['Grade point'] < 2.5).sum()
        else:
            avg_grade = np.nan
            min_grade = np.nan
            num_low_grades = 0
    else:
        avg_grade = np.nan
        min_grade = np.nan
        num_low_grades = 0
    
    return {
        'all_prerequisites_met': all_prerequisites_met,
        'avg_grade_in_prerequisites': avg_grade,
        'min_grade_in_prerequisites': min_grade,
        'num_prerequisites_with_low_grades': num_low_grades
    }

In [76]:
transcript_copy = transcript_df.copy()

transcript_copy['Grade point'] = pd.to_numeric(transcript_copy['Grade point'], errors='coerce')
transcript_copy['Subject Credit'] = pd.to_numeric(transcript_copy['Subject Credit'], errors='coerce')

transcript_copy = transcript_copy.dropna(subset=['Grade point'])
# Normalize course codes in transcript for matching with prerequisites
transcript_copy['Course_Code_Normalized'] = transcript_copy['Course Code'].str.replace(' ', '').str.upper()

print(f"Transcript records after cleaning: {len(transcript_copy)}")
print(f"Unique students: {transcript_copy['StudentRef'].nunique()}")
print(f"Unique courses: {transcript_copy['Course Code'].nunique()}")
transcript_copy.head()

Transcript records after cleaning: 115421
Unique students: 3684
Unique courses: 664


,StudentRef,Admission Year,Program ID,Program,Semester/Year,Academic Year,Student Status,Gender,Course Code,Course Name,...,Grade,Grade point,Grade system,GPA,CGPA,Subject Credit,Calculate toward Graduation Criteria?,Course offering plan name,Student Type,Course_Code_Normalized
0,Sbb69bcfdcfa1e660,2011-2012,BSc-BA,B.Sc - Business Administration,Semester 1,2011-2012,Active,Male,ENGL113,Text and Meaning,...,D,1.0,Migration,1.25,1.25,1.0,Yes,Regular,Regular,ENGL113
1,Sbb69bcfdcfa1e660,2011-2012,BSc-BA,B.Sc - Business Administration,Semester 1,2011-2012,Active,Male,MATH151,Statistics,...,D,1.0,Migration,1.25,1.25,1.0,Yes,Regular,Regular,MATH151
2,Sbb69bcfdcfa1e660,2011-2012,BSc-BA,B.Sc - Business Administration,Semester 1,2011-2012,Active,Male,ENGL111,Expository Writing,...,B,3.0,Migration,1.25,1.25,1.0,Yes,Regular,Regular,ENGL111
3,Sbb69bcfdcfa1e660,2011-2012,BSc-BA,B.Sc - Business Administration,Semester 1,2011-2012,Active,Male,MATH142_A,Calculus I,...,E,0.0,Migration,1.25,1.25,1.0,Yes,Regular,Regular,MATH142_A
4,Sbb69bcfdcfa1e660,2011-2012,BSc-BA,B.Sc - Business Administration,Semester 2,2011-2012,Active,Male,ECON101,Microeconomics,...,E,0.0,Migration,1.21,1.23,1.0,Yes,Regular,Regular,ECON101


In [77]:
student_level_transcript = transcript_copy.groupby('StudentRef').agg({
    'Course Code': 'count',
    'Grade point': ['mean', 'std', 'min', 'max'],
    'Subject Credit': 'sum',
    'Semester/Year': 'nunique'
}).reset_index()

student_level_transcript.columns = ['Student_Ref', 'Total_Courses', 'Avg_Grade_Point', 
                                    'Std_Grade_Point', 'Min_Grade_Point', 'Max_Grade_Point',
                                    'Total_Credits', 'Semesters_Taken']

print(f"Student-level transcript data: {student_level_transcript.shape}")
student_level_transcript.head()

Student-level transcript data: (3684, 8)


,Student_Ref,Total_Courses,Avg_Grade_Point,Std_Grade_Point,Min_Grade_Point,Max_Grade_Point,Total_Credits,Semesters_Taken
0,S00039f6fd1b74390,31,3.645161,0.797847,0.0,4.0,27.5,3
1,S000901505ca1ec7f,36,3.611111,0.728120,0.0,4.0,33.5,2
2,S0021eb5e8ac9bfec,38,3.210526,0.767865,0.0,4.0,35.5,3
3,S0027200343737e85,38,3.236842,1.052410,0.0,4.0,35.5,2
4,S002e2924edb73507,38,3.131579,0.835724,0.0,4.0,35.5,2


## 5. Create Course Difficulty Features

## 5a. Calculate Actual Per-Semester Courses and Credits

**Improvement**: Instead of averaging total courses/credits across semesters (which assumes even distribution), we calculate the actual number of courses and credits taken in each specific semester.

This provides:
- More accurate representation of student workload patterns
- Insights into courseload consistency
- Detection of overloading/underloading patterns

In [ ]:
# Calculate per-semester courses and credits from transcript data
from typing import Any

transcript_for_semester_calc = transcript_copy.copy()
# Add normalized course code to transcript_for_semester_calc if not already present
if 'Course_Code_Normalized' not in transcript_for_semester_calc.columns:
    transcript_for_semester_calc['Course_Code_Normalized'] = transcript_for_semester_calc['Course Code'].str.replace(' ', '').str.upper()
# Map Academic Year and Semester/Year to comparable format
def parse_academic_year_transcript(year_str):
    try:
        return int(str(year_str).split('-')[0])
    except:
        return np.nan

transcript_for_semester_calc['Academic_Year_Numeric'] = transcript_for_semester_calc['Academic Year'].apply(parse_academic_year_transcript)

semester_map = {
    'Semester 1': 1,
    'Semester 2': 2,
    'Semester 3': 3
}
transcript_for_semester_calc['Semester_Numeric'] = transcript_for_semester_calc['Semester/Year'].map(semester_map)

# Sort by student and semester
transcript_for_semester_calc = transcript_for_semester_calc.sort_values(['StudentRef', 'Academic_Year_Numeric', 'Semester_Numeric'])

# Group by student, academic year, and semester to assign semester numbers
transcript_for_semester_calc['semester_key'] = (
    transcript_for_semester_calc['Academic_Year_Numeric'].astype(str) + '_' +
    transcript_for_semester_calc['Semester_Numeric'].astype(str)
)

# For each student, assign semester numbers based on chronological order
semester_number_map = {}
for student_id, group in transcript_for_semester_calc.groupby('StudentRef'):
    unique_semesters = group[['semester_key', 'Academic_Year_Numeric', 'Semester_Numeric']].drop_duplicates().sort_values(['Academic_Year_Numeric', 'Semester_Numeric'])
    for idx, (sem_key, _, _) in enumerate(unique_semesters.values, start=1):
        semester_number_map[(student_id, sem_key)] = idx

transcript_for_semester_calc['semester_number'] = transcript_for_semester_calc.apply(
    lambda row: semester_number_map.get((row['StudentRef'], row['semester_key']), np.nan),
    axis=1
)

# Now calculate courses and credits per semester
semester_courseload = transcript_for_semester_calc.groupby(['StudentRef', 'semester_number']).agg({
    'Course Code': 'count',  # Number of courses
    'Subject Credit': 'sum'   # Total credits
}).reset_index()

semester_courseload.columns = ['Student_Ref', 'semester_number', 'courses_in_semester', 'credits_in_semester']

print(f"Calculated courseload for {len(semester_courseload)} semester records")
print(f"Across {semester_courseload['Student_Ref'].nunique()} unique students")

# Pivot to create semester-by-semester courseload features
semester_courses_pivot = semester_courseload.pivot_table(
    index='Student_Ref',
    columns='semester_number',
    values='courses_in_semester',
    aggfunc='first'
)
semester_courses_pivot.columns = [f'semester_{int(col)}_courses' for col in semester_courses_pivot.columns]

semester_credits_pivot = semester_courseload.pivot_table(
    index='Student_Ref',
    columns='semester_number',
    values='credits_in_semester',
    aggfunc='first'
)
semester_credits_pivot.columns = [f'semester_{int(col)}_credits' for col in semester_credits_pivot.columns]

print(f"\nPivot tables created:")
print(f"  Semester courses: {semester_courses_pivot.shape}")
print(f"  Semester credits: {semester_credits_pivot.shape}")

# Calculate statistics across semesters for each student
courseload_stats = semester_courseload.groupby('Student_Ref').agg({
    'courses_in_semester': ['mean', 'std', 'min', 'max'],
    'credits_in_semester': ['mean', 'std', 'min', 'max']
}).reset_index()

courseload_stats.columns = [
    'Student_Ref',
    'Avg_Courses_Per_Semester', 'Std_Courses_Per_Semester', 'Min_Courses_Per_Semester', 'Max_Courses_Per_Semester',
    'Avg_Credits_Per_Semester', 'Std_Credits_Per_Semester', 'Min_Credits_Per_Semester', 'Max_Credits_Per_Semester'
]

# Fill NaN std with 0 (happens when student has only 1 semester)
courseload_stats['Std_Courses_Per_Semester'] = courseload_stats['Std_Courses_Per_Semester'].fillna(0)
courseload_stats['Std_Credits_Per_Semester'] = courseload_stats['Std_Credits_Per_Semester'].fillna(0)

print(f"\nCourseload statistics calculated for {len(courseload_stats)} students")
print(f"\nSample statistics:")
print(courseload_stats.head())

print(f"\nOverall statistics:")
print(f"  Mean courses per semester: {courseload_stats['Avg_Courses_Per_Semester'].mean():.2f}")
print(f"  Mean credits per semester: {courseload_stats['Avg_Credits_Per_Semester'].mean():.2f}")
print(f"  Mean courseload variability (std courses): {courseload_stats['Std_Courses_Per_Semester'].mean():.2f}")
print(f"  Mean credit variability (std credits): {courseload_stats['Std_Credits_Per_Semester'].mean():.2f}")

Calculated courseload for 26754 semester records
Across 3684 unique students

Pivot tables created:
  Semester courses: (3684, 18)
  Semester credits: (3684, 18)

Courseload statistics calculated for 3684 students

Sample statistics:
         Student_Ref  Avg_Courses_Per_Semester  Std_Courses_Per_Semester  \
0  S00039f6fd1b74390                  5.166667                  0.408248   
1  S000901505ca1ec7f                  4.500000                  0.534522   
2  S0021eb5e8ac9bfec                  4.222222                  0.971825   
3  S0027200343737e85                  4.222222                  1.301708   
4  S002e2924edb73507                  4.750000                  0.707107   

   Min_Courses_Per_Semester  Max_Courses_Per_Semester  \
0                         5                         6   
1                         4                         5   
2                         2                         5   
3                         1                         5   
4                       

In [79]:
course_difficulty = transcript_copy.groupby('Course Code').agg({
    'Grade point': ['mean', 'std', 'count'],
    'Subject Credit': 'first'
}).reset_index()

course_difficulty.columns = ['Course_Code', 'Avg_Grade', 'Std_Grade', 'Enrollment_Count', 'Credits']

course_difficulty['Difficulty_Score'] = 4.0 - course_difficulty['Avg_Grade']

course_difficulty['Difficulty_Category'] = pd.cut(
    course_difficulty['Avg_Grade'],
    bins=[0, 2.5, 3.0, 3.5, 4.5],
    labels=['Very Hard', 'Hard', 'Moderate', 'Easy']
) #based on avg grade hence logic is correct

print(f"Course difficulty metrics created for {len(course_difficulty)} courses")
print("\nDifficulty distribution:")
print(course_difficulty['Difficulty_Category'].value_counts())
print("\nHardest courses:")
print(course_difficulty.nsmallest(10, 'Avg_Grade')[['Course_Code', 'Avg_Grade', 'Difficulty_Category', 'Enrollment_Count']])

Course difficulty metrics created for 664 courses

Difficulty distribution:
Difficulty_Category
Easy         223
Moderate     190
Hard         147
Very Hard     62
Name: count, dtype: int64

Hardest courses:
       Course_Code  Avg_Grade Difficulty_Category  Enrollment_Count
5          9108220        0.0                 NaN                 2
6          9114307        0.0                 NaN                 1
12      AECO 25A20        0.0                 NaN                 1
26      ARCH XB142        0.0                 NaN                 1
31        ARTS 350        0.0                 NaN                 1
32           AS111        0.0                 NaN              2735
34      ASPO 25A22        0.0                 NaN                 1
36      BMET 25A01        0.0                 NaN                 1
135  COMPSCI XB198        0.0                 NaN                 1
202       CSE  355        0.0                 NaN                 3


In [80]:
transcript_with_difficulty = transcript_copy.merge(
    course_difficulty[['Course_Code', 'Difficulty_Score', 'Difficulty_Category']],
    left_on='Course Code',
    right_on='Course_Code',
    how='left'
)

course_features_by_student = transcript_with_difficulty.groupby('StudentRef').agg({
    'Difficulty_Score': ['mean', 'max'],
    'Subject Credit': 'mean'
}).reset_index()

course_features_by_student.columns = ['Student_Ref', 'Avg_Course_Difficulty', 
                                       'Max_Course_Difficulty', 'Avg_Credits_Per_Course']

print(f"Course difficulty features per student: {course_features_by_student.shape}")
course_features_by_student.head()

Course difficulty features per student: (3684, 4)


,Student_Ref,Avg_Course_Difficulty,Max_Course_Difficulty,Avg_Credits_Per_Course
0,S00039f6fd1b74390,1.016160,4.000000,0.887097
1,S000901505ca1ec7f,1.028306,3.991001,0.930556
2,S0021eb5e8ac9bfec,1.052627,4.000000,0.934211
3,S0027200343737e85,1.110678,4.000000,0.934211
4,S002e2924edb73507,0.997727,3.991001,0.934211


In [81]:
# Calculate Program_Avg_Difficulty
# This captures how difficult each program's courses are on average

# Join transcript with course difficulty to get difficulty per enrollment
transcript_with_difficulty_full = transcript_copy.merge(
    course_difficulty[['Course_Code', 'Difficulty_Score']],
    left_on='Course Code',
    right_on='Course_Code',
    how='left'
)

# Calculate average course difficulty per program
program_difficulty = transcript_with_difficulty_full.groupby('Program').agg({
    'Difficulty_Score': 'mean',
    'StudentRef': 'count'  # Number of enrollments
}).reset_index()

program_difficulty.columns = ['Program', 'Program_Avg_Difficulty', 'Total_Enrollments']

print(f"Program difficulty metrics created for {len(program_difficulty)} programs")
print("\nPrograms by average difficulty (hardest first):")
print(program_difficulty.nlargest(10, 'Program_Avg_Difficulty')[['Program', 'Program_Avg_Difficulty', 'Total_Enrollments']])

print("\nPrograms by average difficulty (easiest first):")
print(program_difficulty.nsmallest(10, 'Program_Avg_Difficulty')[['Program', 'Program_Avg_Difficulty', 'Total_Enrollments']])

Program difficulty metrics created for 11 programs

Programs by average difficulty (hardest first):
                                         Program  Program_Avg_Difficulty  \
9                   LLB - Law with Public Policy                1.389687   
3                               B.Sc - Economics                1.192137   
7                B.Sc - Mechatronics Engineering                1.122317   
2                        B.Sc - Computer Science                1.115098   
5          B.Sc - Management Information Systems                1.085551   
0                 B.Sc - Business Administration                1.077967   
1                    B.Sc - Computer Engineering                1.056871   
4   B.Sc - Electrical and Electronic Engineering                1.047821   
6                  B.Sc - Mechanical Engineering                1.002687   
10                                    Undeclared                0.774482   

    Total_Enrollments  
9                 633  
3              

## 6. Merge All Data Sources

In [82]:
master_df = student_level_cgpa.merge(
    student_level_transcript,
    on='Student_Ref',
    how='left'
)

master_df = master_df.merge(
    course_features_by_student,
    on='Student_Ref',
    how='left'
)

master_df = master_df.merge(
    courseload_stats,
    on='Student_Ref',
    how='left'
)

master_df = master_df.merge(
    semester_features,
    left_on='Student_Ref',
    right_index=True,
    how='left'
)

print(f"Master dataset created: {master_df.shape}")
print(f"Total features: {len(master_df.columns)}")
print(f"\nColumn groups:")
print(f"  Demographic: Gender, Nationality, Program, etc.")
print(f"  Performance: CGPA, GPA stats, grades")
print(f"  Course: Total courses, credits, difficulty")
print(f"  Courseload: Avg/Std/Min/Max courses and credits per semester")
print(f"  Semester: semester_1_gpa, semester_2_gpa, ...")

Master dataset created: (3718, 70)
Total features: 70

Column groups:
  Demographic: Gender, Nationality, Program, etc.
  Performance: CGPA, GPA stats, grades
  Course: Total courses, credits, difficulty
  Courseload: Avg/Std/Min/Max courses and credits per semester
  Semester: semester_1_gpa, semester_2_gpa, ...


## 7. Create Base Features

In [84]:
master_df['Is_Local'] = (master_df['Nationality'] == 'Country0').astype(int)
master_df['Is_International'] = 1 - master_df['Is_Local']

print("Nationality distribution:")
print(f"  Local (Country0): {master_df['Is_Local'].sum()}")
print(f"  International: {master_df['Is_International'].sum()}")

master_df['Application_Category'] = master_df['Application_Category'].fillna('Unknown')

# Need-based scholarship: MCF only
master_df['Has_Need_Based_Scholarship'] = master_df['Application_Category'].str.contains(
    'MCF', case=False, na=False
).astype(int)

# Any scholarship (for filtering unknown status)
scholarship_keywords = ['afa', 'mcf', 'ffp', 'other', 'withdrawn']
master_df['Has_Any_Scholarship'] = master_df['Application_Category'].str.lower().apply(
    lambda x: int(any(keyword in str(x) for keyword in scholarship_keywords))
)

# Non-need-based scholarship: All scholarships except MCF
master_df['Has_Non_Need_Based_Scholarship'] = (
    (master_df['Has_Any_Scholarship'] == 1) & 
    (master_df['Has_Need_Based_Scholarship'] == 0)
).astype(int)

print("\nScholarship distribution:")
print(f"  Need-based (MCF): {master_df['Has_Need_Based_Scholarship'].sum()}")
print(f"  Non-need-based (AFA, FFP, Other): {master_df['Has_Non_Need_Based_Scholarship'].sum()}")
print(f"  Total scholarships: {master_df['Has_Any_Scholarship'].sum()}")
print(f"  No scholarship: {(master_df['Has_Any_Scholarship'] == 0).sum()}") #unknown category



Nationality distribution:
  Local (Country0): 2944
  International: 774

Scholarship distribution:
  Need-based (MCF): 616
  Non-need-based (AFA, FFP, Other): 2122
  Total scholarships: 2738
  No scholarship: 980


In [85]:
master_df['Is_Male'] = (master_df['Gender'] == 'Male').astype(int)
master_df['Is_Female'] = 1 - master_df['Is_Male']

stem_programs = ['Computer Science', 'Engineering', 'Mechatronics', 'Electrical', 'Mechanical']
master_df['Is_STEM'] = master_df['Program'].apply(
    lambda x: int(any(prog in str(x) for prog in stem_programs))
)

business_programs = ['Business Administration', 'Management Information Systems']
master_df['Is_Business'] = master_df['Program'].apply(
    lambda x: int(any(prog in str(x) for prog in business_programs))
)

master_df['Graduated'] = (master_df['Student_Status'] == 'Graduated').astype(int)
master_df['Active'] = (master_df['Student_Status'] == 'Active').astype(int)
master_df['Exited'] = master_df['Student_Status'].apply(
    lambda x: int(x not in ['Graduated', 'Active'])
)

print("Base features created:")
print(f"  Gender - Male: {master_df['Is_Male'].sum()}, Female: {master_df['Is_Female'].sum()}")
print(f"  Program - STEM: {master_df['Is_STEM'].sum()}, Business: {master_df['Is_Business'].sum()}")
print(f"  Status - Graduated: {master_df['Graduated'].sum()}, Active: {master_df['Active'].sum()}, Exited: {master_df['Exited'].sum()}")

Base features created:
  Gender - Male: 2072, Female: 1646
  Program - STEM: 1684, Business: 1916
  Status - Graduated: 1956, Active: 1514, Exited: 248


## 8. Engineer Trajectory Features

In [86]:
semester_gpa_cols = [col for col in master_df.columns if 'semester_' in col and '_gpa' in col and 'cgpa' not in col]
semester_gpa_cols = sorted(semester_gpa_cols, key=lambda x: int(x.split('_')[1]))

print(f"Found {len(semester_gpa_cols)} semester GPA columns")

def calculate_gpa_slope(row):
    gpas = row[semester_gpa_cols].dropna().values
    if len(gpas) < 2:
        return 0
    # Convert to numeric and filter out any NaN or inf values
    gpas = pd.to_numeric(gpas, errors='coerce')
    gpas = gpas[~np.isnan(gpas)]
    if len(gpas) < 2:
        return 0
    try:
        x = np.arange(len(gpas))
        slope, _, _, _, _ = stats.linregress(x, gpas)
        return slope if not np.isnan(slope) else 0
    except:
        return 0

def count_improvements(row):
    gpas = row[semester_gpa_cols].dropna().values
    gpas = pd.to_numeric(gpas, errors='coerce')
    gpas = gpas[~np.isnan(gpas)]
    if len(gpas) < 2:
        return 0
    return np.sum(np.diff(gpas) > 0.1)

def count_declines(row):
    gpas = row[semester_gpa_cols].dropna().values
    gpas = pd.to_numeric(gpas, errors='coerce')
    gpas = gpas[~np.isnan(gpas)]
    if len(gpas) < 2:
        return 0
    return np.sum(np.diff(gpas) < -0.1)

master_df['GPA_Slope'] = master_df.apply(calculate_gpa_slope, axis=1)
master_df['Num_Improvements'] = master_df.apply(count_improvements, axis=1)
master_df['Num_Declines'] = master_df.apply(count_declines, axis=1)

master_df['GPA_Volatility'] = master_df[semester_gpa_cols].std(axis=1)
master_df['GPA_Range'] = master_df['Max_GPA'] - master_df['Min_GPA']

first_semester_gpa = master_df['semester_1_gpa']
last_semester_gpa = master_df[semester_gpa_cols].apply(lambda x: x.dropna().iloc[-1] if len(x.dropna()) > 0 else np.nan, axis=1)
master_df['GPA_Overall_Improvement'] = last_semester_gpa - first_semester_gpa

print("Trajectory features created:")
print(f"  GPA_Slope - mean: {master_df['GPA_Slope'].mean():.4f}")
print(f"  GPA_Volatility - mean: {master_df['GPA_Volatility'].mean():.3f}")
print(f"  GPA_Overall_Improvement - mean: {master_df['GPA_Overall_Improvement'].mean():.3f}")

Found 19 semester GPA columns
Trajectory features created:
  GPA_Slope - mean: -0.0059
  GPA_Volatility - mean: 0.415
  GPA_Overall_Improvement - mean: 0.077


In [87]:
master_df['Years_to_Complete'] = master_df['Grad_Year'] - master_df['Admission_Year']
master_df['On_Time_Graduation'] = (
    (master_df['Graduated'] == 1) & 
    (master_df['Years_to_Complete'] <= 4)
).astype(int)

master_df['Extended_Time'] = (master_df['Years_to_Complete'] > 4).astype(int)

print("Progression features created")
print(f"\nGraduation statistics:")
print(f"  Mean years to complete: {master_df[master_df['Graduated']==1]['Years_to_Complete'].mean():.2f}")
print(f"  On-time graduation rate: {master_df['On_Time_Graduation'].mean():.1%}")
print(f"  Extended time rate: {master_df[master_df['Graduated']==1]['Extended_Time'].mean():.1%}")

print(f"\nCourseload statistics ")
print(f"  Mean avg courses per semester: {master_df['Avg_Courses_Per_Semester'].mean():.2f}")
print(f"  Mean avg credits per semester: {master_df['Avg_Credits_Per_Semester'].mean():.2f}")
print(f"  Mean courseload variability (std): {master_df['Std_Courses_Per_Semester'].mean():.2f}")
print(f"  Students who overloaded (max > avg + 2 std): {(master_df['Max_Courses_Per_Semester'] > master_df['Avg_Courses_Per_Semester'] + 2*master_df['Std_Courses_Per_Semester']).sum()}")

Progression features created

Graduation statistics:
  Mean years to complete: 3.99
  On-time graduation rate: 52.6%
  Extended time rate: 0.0%

Courseload statistics 
  Mean avg courses per semester: 4.44
  Mean avg credits per semester: 4.00
  Mean courseload variability (std): 0.90
  Students who overloaded (max > avg + 2 std): 87


## 9. Create Target Variables

In [88]:
def categorize_cgpa(cgpa):
    if pd.isna(cgpa):
        return 'Unknown'
    elif cgpa >= 3.5:
        return 'Deans_List'
    elif cgpa >= 2.5:
        return 'Good_Standing'
    elif cgpa >= 2.0:
        return 'At_Risk'
    else:
        return 'Probation'

master_df['Grade_Bin'] = master_df['Final_CGPA'].apply(categorize_cgpa)

grade_bin_numeric = {
    'Probation': 0,
    'At_Risk': 1,
    'Good_Standing': 2,
    'Deans_List': 3,
    'Unknown': np.nan
}
master_df['Grade_Bin_Numeric'] = master_df['Grade_Bin'].map(grade_bin_numeric)

master_df['Is_Probation'] = (master_df['Final_CGPA'] < 2.0).astype(int)
master_df['Is_At_Risk'] = ((master_df['Final_CGPA'] >= 2.0) & (master_df['Final_CGPA'] < 2.5)).astype(int)
master_df['Is_Deans_List'] = (master_df['Final_CGPA'] >= 3.5).astype(int)

print("Target variables created")
print("\nGrade Bin Distribution:")
print(master_df['Grade_Bin'].value_counts())
print(f"\nProbation cases: {master_df['Is_Probation'].sum()} ({master_df['Is_Probation'].mean():.1%})")
print(f"At-risk cases: {master_df['Is_At_Risk'].sum()} ({master_df['Is_At_Risk'].mean():.1%})")
print(f"Dean's List: {master_df['Is_Deans_List'].sum()} ({master_df['Is_Deans_List'].mean():.1%})")

Target variables created

Grade Bin Distribution:
Grade_Bin
Good_Standing    2327
Deans_List        771
At_Risk           444
Probation         176
Name: count, dtype: int64

Probation cases: 176 (4.7%)
At-risk cases: 444 (11.9%)
Dean's List: 771 (20.7%)


In [89]:
master_df['Retention_Status'] = master_df['Student_Status'].apply(
    lambda x: 'Retained' if x in ['Graduated', 'Active'] else 'Dropped_Out'
)
master_df['Dropped_Out'] = (master_df['Retention_Status'] == 'Dropped_Out').astype(int)

print("Retention target created")
print(f"\nRetention statistics:")
print(master_df['Retention_Status'].value_counts())
print(f"\nDropout rate: {master_df['Dropped_Out'].mean():.1%}")

Retention target created

Retention statistics:
Retention_Status
Retained       3470
Dropped_Out     248
Name: count, dtype: int64

Dropout rate: 6.7%


## 10. Save Master Dataset

In [90]:
master_df.to_csv('master_dataset.csv', index=False)
print(f"Master dataset saved: {master_df.shape}")
print(f"Total features: {len(master_df.columns)}")
print(f"\nKey feature groups:")
print(f"  Semester GPAs: {len([c for c in master_df.columns if 'semester_' in c])}")
print(f"  Demographics: Gender, Nationality, Program, Scholarship")
print(f"  Trajectory: Slope, Volatility, Improvements/Declines")
print(f"  Targets: Grade_Bin, Probation, Graduation, Retention")

Master dataset saved: (3718, 98)
Total features: 98

Key feature groups:
  Semester GPAs: 38
  Demographics: Gender, Nationality, Program, Scholarship
  Trajectory: Slope, Volatility, Improvements/Declines
  Targets: Grade_Bin, Probation, Graduation, Retention


## 11. Prepare Model-Specific Datasets

Now we'll create separate datasets for each supervised learning task.

### 11.1 Early Warning System - First Semester

In [91]:
# For Early Warning, we need to add course difficulty and courseload features
# These are known at prediction time and provide crucial context

# Get program difficulty for each student
master_df_with_prog_diff = master_df.merge(
    program_difficulty[['Program', 'Program_Avg_Difficulty']],
    on='Program',
    how='left'
)

# Calculate course difficulty metrics for first semester from transcript
first_sem_difficulty = []

for student_id in master_df['Student_Ref']:
    # Get courses taken in first semester
    student_transcript = transcript_for_semester_calc[
        (transcript_for_semester_calc['StudentRef'] == student_id) & 
        (transcript_for_semester_calc['semester_number'] == 1)
    ]
    
    if len(student_transcript) > 0:
        # Merge with difficulty
        student_courses_with_diff = student_transcript.merge(
            course_difficulty[['Course_Code', 'Difficulty_Score']],
            left_on='Course Code',
            right_on='Course_Code',
            how='left'
        )
        
        avg_diff = student_courses_with_diff['Difficulty_Score'].mean()
        max_diff = student_courses_with_diff['Difficulty_Score'].max()
    else:
        avg_diff = np.nan
        max_diff = np.nan
    
    first_sem_difficulty.append({
        'Student_Ref': student_id,
        'first_sem_avg_difficulty': avg_diff,
        'first_sem_max_difficulty': max_diff
    })

first_sem_diff_df = pd.DataFrame(first_sem_difficulty)

# Merge everything
early_warning_1st_sem = master_df_with_prog_diff[master_df_with_prog_diff['semester_1_gpa'].notna()].copy()
early_warning_1st_sem = early_warning_1st_sem.merge(first_sem_diff_df, on='Student_Ref', how='left')

# Get courseload for semester 1
semester_1_courseload = semester_courseload[semester_courseload['semester_number'] == 1][['Student_Ref', 'courses_in_semester', 'credits_in_semester']].copy()
semester_1_courseload.columns = ['Student_Ref', 'first_sem_courses', 'first_sem_credits']
early_warning_1st_sem = early_warning_1st_sem.merge(semester_1_courseload, on='Student_Ref', how='left')

# One-hot encode program
early_warning_1st_sem_encoded = pd.get_dummies(early_warning_1st_sem, columns=['Program'], prefix='Program')
program_cols_early = [col for col in early_warning_1st_sem_encoded.columns if col.startswith('Program_')]

features_1st_sem = [
    'semester_1_gpa',
    'first_sem_courses', 'first_sem_credits',  # NEW: Actual courseload
    'first_sem_avg_difficulty', 'first_sem_max_difficulty',  # NEW: Course difficulty
    'Program_Avg_Difficulty',  # NEW: Program difficulty
    'Is_Local', 'Is_Male', 'Is_STEM', 'Is_Business',
    'Has_Need_Based_Scholarship', 'Has_Non_Need_Based_Scholarship',
    'Admission_Year'
] + program_cols_early  # NEW: Program one-hot encoded

X_early_1st = early_warning_1st_sem_encoded[features_1st_sem]
y_early_1st_multiclass = early_warning_1st_sem_encoded['Grade_Bin']
y_early_1st_numeric = early_warning_1st_sem_encoded['Grade_Bin_Numeric']
y_early_1st_cgpa = early_warning_1st_sem_encoded['Final_CGPA']

early_warning_1st_sem_data = {
    'X': X_early_1st,
    'y_multiclass': y_early_1st_multiclass,
    'y_numeric': y_early_1st_numeric,
    'y_cgpa': y_early_1st_cgpa,
    'student_ids': early_warning_1st_sem_encoded['Student_Ref']
}

print("Early Warning System - First Semester")
print(f"  Dataset size: {len(X_early_1st)}")
print(f"  Features: {len(features_1st_sem)}")
print(f"    - Base features: 12")
print(f"    - Program one-hot encoded: {len(program_cols_early)} programs")
print(f"    - NEW: Includes courseload, course difficulty, and program features")
print(f"  Target distribution:")
print(y_early_1st_multiclass.value_counts())
print(f"\n  Sample feature values:")
print(f"    Mean first sem courses: {early_warning_1st_sem_encoded['first_sem_courses'].mean():.2f}")
print(f"    Mean first sem credits: {early_warning_1st_sem_encoded['first_sem_credits'].mean():.2f}")
print(f"    Mean course difficulty: {early_warning_1st_sem_encoded['first_sem_avg_difficulty'].mean():.2f}")

Early Warning System - First Semester
  Dataset size: 3718
  Features: 25
    - Base features: 12
    - Program one-hot encoded: 12 programs
    - NEW: Includes courseload, course difficulty, and program features
  Target distribution:
Grade_Bin
Good_Standing    2327
Deans_List        771
At_Risk           444
Probation         176
Name: count, dtype: int64

  Sample feature values:
    Mean first sem courses: 5.00
    Mean first sem credits: 3.97
    Mean course difficulty: 1.64


### 11.2 Early Warning System - First Year

In [92]:
# Calculate course difficulty metrics for first year (semesters 1 & 2) from transcript
first_year_difficulty = []

for student_id in master_df['Student_Ref']:
    # Get courses taken in first year (semesters 1 and 2)
    student_transcript = transcript_for_semester_calc[
        (transcript_for_semester_calc['StudentRef'] == student_id) & 
        (transcript_for_semester_calc['semester_number'].isin([1, 2]))
    ]
    
    if len(student_transcript) > 0:
        # Merge with difficulty
        student_courses_with_diff = student_transcript.merge(
            course_difficulty[['Course_Code', 'Difficulty_Score']],
            left_on='Course Code',
            right_on='Course_Code',
            how='left'
        )
        
        avg_diff = student_courses_with_diff['Difficulty_Score'].mean()
        max_diff = student_courses_with_diff['Difficulty_Score'].max()
    else:
        avg_diff = np.nan
        max_diff = np.nan
    
    first_year_difficulty.append({
        'Student_Ref': student_id,
        'first_year_avg_difficulty': avg_diff,
        'first_year_max_difficulty': max_diff
    })

first_year_diff_df = pd.DataFrame(first_year_difficulty)

# Start with students who have both semester 1 and 2 data
early_warning_1st_year = master_df_with_prog_diff[
    master_df_with_prog_diff['semester_1_gpa'].notna() & 
    master_df_with_prog_diff['semester_2_gpa'].notna()
].copy()

# Add calculated GPA features
early_warning_1st_year['First_Year_Mean_GPA'] = early_warning_1st_year[['semester_1_gpa', 'semester_2_gpa']].mean(axis=1)
early_warning_1st_year['First_Year_GPA_Change'] = early_warning_1st_year['semester_2_gpa'] - early_warning_1st_year['semester_1_gpa']
early_warning_1st_year['First_Year_GPA_Std'] = early_warning_1st_year[['semester_1_gpa', 'semester_2_gpa']].std(axis=1)

# Merge difficulty metrics
early_warning_1st_year = early_warning_1st_year.merge(first_year_diff_df, on='Student_Ref', how='left')

# Get courseload for first year (sum of semesters 1 and 2)
first_year_courseload = semester_courseload[semester_courseload['semester_number'].isin([1, 2])].groupby('Student_Ref').agg({
    'courses_in_semester': 'sum',
    'credits_in_semester': 'sum'
}).reset_index()
first_year_courseload.columns = ['Student_Ref', 'first_year_courses', 'first_year_credits']

# Also get per-semester averages for first year
first_year_courseload_avg = semester_courseload[semester_courseload['semester_number'].isin([1, 2])].groupby('Student_Ref').agg({
    'courses_in_semester': 'mean',
    'credits_in_semester': 'mean'
}).reset_index()
first_year_courseload_avg.columns = ['Student_Ref', 'first_year_avg_courses_per_sem', 'first_year_avg_credits_per_sem']

early_warning_1st_year = early_warning_1st_year.merge(first_year_courseload, on='Student_Ref', how='left')
early_warning_1st_year = early_warning_1st_year.merge(first_year_courseload_avg, on='Student_Ref', how='left')

# One-hot encode program
early_warning_1st_year_encoded = pd.get_dummies(early_warning_1st_year, columns=['Program'], prefix='Program')
program_cols_1st_year = [col for col in early_warning_1st_year_encoded.columns if col.startswith('Program_')]

features_1st_year = [
    'semester_1_gpa', 'semester_2_gpa',
    'First_Year_Mean_GPA', 'First_Year_GPA_Change', 'First_Year_GPA_Std',
    'first_year_courses', 'first_year_credits',  # NEW: Total courseload in first year
    'first_year_avg_courses_per_sem', 'first_year_avg_credits_per_sem',  # NEW: Average per semester
    'first_year_avg_difficulty', 'first_year_max_difficulty',  # NEW: Course difficulty
    'Program_Avg_Difficulty',  # NEW: Program difficulty
    'Is_Local', 'Is_Male', 'Is_STEM', 'Is_Business',
    'Has_Need_Based_Scholarship', 'Has_Non_Need_Based_Scholarship',
    'Admission_Year'
] + program_cols_1st_year  # NEW: Program one-hot encoded

X_early_1st_year = early_warning_1st_year_encoded[features_1st_year]
y_early_1st_year_multiclass = early_warning_1st_year_encoded['Grade_Bin']
y_early_1st_year_numeric = early_warning_1st_year_encoded['Grade_Bin_Numeric']
y_early_1st_year_cgpa = early_warning_1st_year_encoded['Final_CGPA']

early_warning_1st_year_data = {
    'X': X_early_1st_year,
    'y_multiclass': y_early_1st_year_multiclass,
    'y_numeric': y_early_1st_year_numeric,
    'y_cgpa': y_early_1st_year_cgpa,
    'student_ids': early_warning_1st_year_encoded['Student_Ref']
}

print("Early Warning System - First Year")
print(f"  Dataset size: {len(X_early_1st_year)}")
print(f"  Features: {len(features_1st_year)}")
print(f"    - Base features: 16")
print(f"    - Program one-hot encoded: {len(program_cols_1st_year)} programs")
print(f"    - NEW: Includes courseload, course difficulty, and program features")
print(f"  Target distribution:")
print(y_early_1st_year_multiclass.value_counts())
print(f"\n  Sample feature values:")
print(f"    Mean first year courses: {early_warning_1st_year_encoded['first_year_courses'].mean():.2f}")
print(f"    Mean first year credits: {early_warning_1st_year_encoded['first_year_credits'].mean():.2f}")
print(f"    Mean course difficulty: {early_warning_1st_year_encoded['first_year_avg_difficulty'].mean():.2f}")

Early Warning System - First Year
  Dataset size: 3618
  Features: 30
    - Base features: 16
    - Program one-hot encoded: 11 programs
    - NEW: Includes courseload, course difficulty, and program features
  Target distribution:
Grade_Bin
Good_Standing    2297
Deans_List        740
At_Risk           441
Probation         140
Name: count, dtype: int64

  Sample feature values:
    Mean first year courses: 9.88
    Mean first year credits: 8.35
    Mean course difficulty: 1.33


### 11.3 Semester-by-Semester Probation Prediction

**Objective**: Predict if a student will be on probation (CGPA < 2.0) at the end of a given semester.

**Key Correction**: Probation is determined at the END of each semester, not based on final CGPA only. This creates ~24K semester-level records instead of 176 student-level records.

**Features**: Cumulative performance up to current semester, trajectory, engagement metrics

In [93]:
# Create semester-level dataset for probation prediction
# CRITICAL: Track ALL semester instances, not just first probation
# Students can recover from probation and fall back again, so we need to predict EVERY semester
# Get program difficulty for each student

probation_records = []

for idx, row in cgpa_copy.iterrows():
    student_id = row['Student Ref']
    sem_num = row['semester_number']
    current_cgpa = row['CGPA']
    current_gpa = row['GPA']
    
    # Skip first semester (need prior data for predictions)
    if sem_num == 1 or pd.isna(current_cgpa) or pd.isna(current_gpa):
        continue
    
    # Get student-level features from master_df
    student_data = master_df[master_df['Student_Ref'] == student_id]
    if len(student_data) == 0:
        continue
    student_data = student_data.iloc[0]

    # Get program difficulty for student's program
    student_program = student_data['Program']
    prog_diff_data = program_difficulty[program_difficulty['Program'] == student_program]
    if len(prog_diff_data) > 0:
        program_avg_diff = prog_diff_data.iloc[0]['Program_Avg_Difficulty']
    else:
        program_avg_diff = 1.0  # Default if program not found

    # Get prior semester GPAs and CGPAs
    prior_gpa_cols = [f'semester_{i}_gpa' for i in range(1, sem_num)]
    prior_gpas = [student_data[col] for col in prior_gpa_cols if col in student_data and pd.notna(student_data[col])]
    
    prior_cgpa_cols = [f'semester_{i}_cgpa' for i in range(1, sem_num)]
    prior_cgpas = [student_data[col] for col in prior_cgpa_cols if col in student_data and pd.notna(student_data[col])]
    
    if len(prior_gpas) == 0 or len(prior_cgpas) == 0:
        continue
    
    # Calculate trajectory features from PRIOR semesters only
    mean_prior_gpa = np.mean(prior_gpas)
    std_prior_gpa = np.std(prior_gpas) if len(prior_gpas) > 1 else 0
    min_prior_gpa = np.min(prior_gpas)
    trend_prior = np.mean(np.diff(prior_gpas)) if len(prior_gpas) > 1 else 0
    
    # Probation history features - TRACK ALL INSTANCES
    num_prior_probations = sum(1 for cgpa in prior_cgpas if cgpa < 2.0)
    ever_on_probation = int(num_prior_probations > 0)
    
    # Was student on probation LAST semester? (critical for consecutive probation tracking)
    last_sem_on_probation = int(prior_cgpas[-1] < 2.0) if len(prior_cgpas) > 0 else 0
    
    # Has student recovered from probation before?
    recovered_before = 0
    if num_prior_probations > 0:
        # Check if any semester after first probation had CGPA >= 2.0
        first_prob_idx = next(i for i, cgpa in enumerate(prior_cgpas) if cgpa < 2.0)
        if any(cgpa >= 2.0 for cgpa in prior_cgpas[first_prob_idx+1:]):
            recovered_before = 1
    
    # Most recent CGPA (end of last semester)
    last_cgpa = prior_cgpas[-1]
    
    # NEW: Get actual courseload for prior semesters
    # Get courseload from semester_courseload for prior semesters
    student_courseload = semester_courseload[semester_courseload['Student_Ref'] == student_id]
    prior_courseloads = student_courseload[student_courseload['semester_number'] < sem_num]
    
    if len(prior_courseloads) > 0:
        avg_prior_courses = prior_courseloads['courses_in_semester'].mean()
        avg_prior_credits = prior_courseloads['credits_in_semester'].mean()
        std_prior_courses = prior_courseloads['courses_in_semester'].std() if len(prior_courseloads) > 1 else 0
        max_prior_courses = prior_courseloads['courses_in_semester'].max()
    else:
        # Fallback to overall statistics if no prior semester data
        avg_prior_courses = student_data.get('Avg_Courses_Per_Semester', 5)
        avg_prior_credits = student_data.get('Avg_Credits_Per_Semester', 4.5)
        std_prior_courses = 0
        max_prior_courses = avg_prior_courses
    
    # Target: Will student be on probation THIS semester?
    on_probation = int(current_cgpa < 2.0)
    
    record = {
        'Student_Ref': student_id,
        'semester_number': sem_num,
        'mean_prior_gpa': mean_prior_gpa,
        'std_prior_gpa': std_prior_gpa,
        'min_prior_gpa': min_prior_gpa,
        'trend_prior_gpa': trend_prior,
        'last_semester_gpa': prior_gpas[-1],
        'last_semester_cgpa': last_cgpa,
        'semesters_completed': len(prior_gpas),
        'num_prior_probations': num_prior_probations,
        'ever_on_probation_before': ever_on_probation,
        'last_sem_on_probation': last_sem_on_probation,
        'recovered_from_probation_before': recovered_before,
        'avg_prior_courses_per_sem': avg_prior_courses,  # NEW: Actual courseload
        'avg_prior_credits_per_sem': avg_prior_credits,  # NEW
        'std_prior_courses': std_prior_courses,  # NEW: Courseload variability
        'max_prior_courses': max_prior_courses,  # NEW: Peak courseload
        'Is_Local': student_data['Is_Local'],
        'Is_Male': student_data['Is_Male'],
        'Is_STEM': student_data['Is_STEM'],
        'Has_Any_Scholarship': student_data['Has_Any_Scholarship'],
        'Admission_Year': student_data['Admission_Year'],
        'on_probation': on_probation,
        'Program_Avg_Difficulty': program_avg_diff  # Add program difficulty
    }
    probation_records.append(record)

probation_df = pd.DataFrame(probation_records)


probation_features = [
    'mean_prior_gpa', 'std_prior_gpa', 'min_prior_gpa', 'trend_prior_gpa',
    'last_semester_gpa', 'last_semester_cgpa', 'semesters_completed',
    'num_prior_probations', 'ever_on_probation_before', 'last_sem_on_probation',
    'recovered_from_probation_before',
    'avg_prior_courses_per_sem', 'avg_prior_credits_per_sem',  # NEW
    'std_prior_courses', 'max_prior_courses',  # NEW
    'Is_Local', 'Is_Male', 'Is_STEM', 'Has_Any_Scholarship',
    'Admission_Year',
    'Program_Avg_Difficulty'
]

X_probation = probation_df[probation_features]
y_probation = probation_df['on_probation']

probation_dataset = {
    'X': X_probation,
    'y': y_probation,
    'student_ids': probation_df['Student_Ref'],
    'semester_numbers': probation_df['semester_number']
}

print("Semester-by-Semester Probation Prediction")
print(f"  Dataset size: {len(X_probation)} semester records")
print(f"  From {probation_df['Student_Ref'].nunique()} unique students")
print(f"  Features: {len(probation_features)}")
print(f"    - Includes NEW actual courseload features (not averaged)")
print(f"  Target distribution:")
print(f"    On Probation (CGPA < 2.0): {y_probation.sum()} ({y_probation.mean():.1%})")
print(f"    Not on Probation: {(y_probation == 0).sum()} ({(y_probation == 0).mean():.1%})")

print(f"\n  Probation history breakdown:")
print(f"    Never on probation before: {(probation_df['num_prior_probations'] == 0).sum()}")
print(f"    On probation 1+ times before: {(probation_df['num_prior_probations'] > 0).sum()}")
print(f"    Recovered from probation before: {probation_df['recovered_from_probation_before'].sum()}")
print(f"    On probation last semester: {probation_df['last_sem_on_probation'].sum()}")

print(f"\n  TRACKS ALL INSTANCES: Students can fall into probation, recover, and fall again")
print(f"  CRITICAL: Enables semester-by-semester early intervention")

Semester-by-Semester Probation Prediction
  Dataset size: 20930 semester records
  From 3618 unique students
  Features: 21
    - Includes NEW actual courseload features (not averaged)
  Target distribution:
    On Probation (CGPA < 2.0): 810 (3.9%)
    Not on Probation: 20120 (96.1%)

  Probation history breakdown:
    Never on probation before: 18751
    On probation 1+ times before: 2179
    Recovered from probation before: 1376
    On probation last semester: 937

  TRACKS ALL INSTANCES: Students can fall into probation, recover, and fall again
  CRITICAL: Enables semester-by-semester early intervention


### 11.5 STEM Course Success Predictor

**Objective**: Predict if a student will pass a STEM course (Grade ≥ 2.0) based on their profile and the course characteristics.

**STEM Courses Include**: CS, COMP, MATH, ENGR, MECH, ELEC, PHYS, CHEM

In [67]:
# Filter for STEM courses
stem_pattern = r'CS |COMP|MATH|ENGR|MECH|ELEC|PHYS|CHEM'
stem_courses = transcript_copy[transcript_copy['Course Code'].str.contains(stem_pattern, case=False, na=False)].copy()

print(f"STEM course enrollments: {len(stem_courses)}")
print(f"Unique STEM courses: {stem_courses['Course Code'].nunique()}")
print(f"Students who took STEM courses: {stem_courses['StudentRef'].nunique()}")

# Merge with course difficulty
stem_courses_with_difficulty = stem_courses.merge(
    course_difficulty[['Course_Code', 'Difficulty_Score', 'Avg_Grade']],
    left_on='Course Code',
    right_on='Course_Code',
    how='left'
)

# This gives us the semester when each course was taken
stem_courses_with_difficulty = stem_courses_with_difficulty.merge(
    transcript_for_semester_calc[['StudentRef', 'Course_Code_Normalized', 'semester_number']].drop_duplicates(),
    left_on=['StudentRef', 'Course_Code_Normalized'],
    right_on=['StudentRef', 'Course_Code_Normalized'],
    how='left'
)

print(f"\\nAdded semester_number to STEM courses dataset")
print(f"Courses with semester info: {stem_courses_with_difficulty['semester_number'].notna().sum()}")

# For each enrollment, get student's state BEFORE taking the course + prerequisite features
stem_course_records = []

print("\\nCalculating features for each STEM course enrollment (including prerequisites)...")
print("This may take a few minutes...")

for idx, row in stem_courses_with_difficulty.iterrows():
    if idx % 5000 == 0:
        print(f"  Processed {idx}/{len(stem_courses_with_difficulty)} enrollments...")
    
    student_id = row['StudentRef']
    course_code = row['Course Code']
    course_code_norm = row['Course_Code_Normalized']
    grade_point = row['Grade point']
    student_program = row['Program'] 
    enrollment_sem = row['semester_number']  # FIX: Now this exists!
    
    # Skip if we don't have semester information
    if pd.isna(enrollment_sem):
        continue
    
    # Get student's data from master_df
    student_data = master_df[master_df['Student_Ref'] == student_id]
    if len(student_data) == 0:
        continue
    
    student_data = student_data.iloc[0]
    # student_program = student_data['Program']
    
    # Count prior courses in same department
    course_dept = course_code.split()[0] if ' ' in course_code else course_code[:4]
    prior_dept_courses = stem_courses_with_difficulty[
        (stem_courses_with_difficulty['StudentRef'] == student_id) &
        (stem_courses_with_difficulty['Course Code'].str.startswith(course_dept))
    ]
    num_prior_dept_courses = len(prior_dept_courses) - 1  # Exclude current course
    
    # Average grade in prior department courses
    if num_prior_dept_courses > 0:
        avg_prior_dept_grade = prior_dept_courses.iloc[:-1]['Grade point'].mean()
    else:
        avg_prior_dept_grade = student_data['Mean_GPA']  # Use overall GPA as fallback
    
    # Get program difficulty for student's program
    prog_diff_data = program_difficulty[program_difficulty['Program'] == student_program]
    if len(prog_diff_data) > 0:
        program_avg_diff = prog_diff_data.iloc[0]['Program_Avg_Difficulty']
    else:
        program_avg_diff = 1.0  # Default if program not found
    
    # NEW: Calculate prerequisite features
    prereq_features = calculate_prerequisite_features(
        student_id=student_id,
        course_code=course_code_norm,
        program=student_program,
        enrollment_semester=enrollment_sem,  
        transcript_df=transcript_for_semester_calc,
        prereq_lookup=prereq_df
    )

    record = {
        'Student_Ref': student_id,
        'Course_Code': course_code,
        'Student_Program': student_program,
        'Program_Avg_Difficulty': program_avg_diff,
        'Student_Mean_GPA': student_data['Mean_GPA'],
        'Student_CGPA': student_data['Final_CGPA'],
        'Course_Difficulty': row['Difficulty_Score'],
        'Course_Avg_Grade': row['Avg_Grade'],
        'Num_Prior_Dept_Courses': num_prior_dept_courses,
        'Avg_Prior_Dept_Grade': avg_prior_dept_grade,
        'Total_Courses_Taken': student_data['Total_Courses'],
        'Avg_Courses_Per_Semester': student_data.get('Avg_Courses_Per_Semester', 5),
        'Std_Courses_Per_Semester': student_data.get('Std_Courses_Per_Semester', 0),
        'Max_Courses_Per_Semester': student_data.get('Max_Courses_Per_Semester', 5),
        'Is_Local': student_data['Is_Local'],
        'Is_Male': student_data['Is_Male'],
        'Is_STEM_Major': student_data['Is_STEM'],
        'Has_Any_Scholarship': student_data['Has_Any_Scholarship'],
        'Admission_Year': student_data['Admission_Year'],
        # NEW: Prerequisite features
        'all_prerequisites_met': prereq_features['all_prerequisites_met'],
        'avg_grade_in_prerequisites': prereq_features['avg_grade_in_prerequisites'],
        'min_grade_in_prerequisites': prereq_features['min_grade_in_prerequisites'],
        'num_prerequisites_with_low_grades': prereq_features['num_prerequisites_with_low_grades'],
        'Passed': int(grade_point >= 2.0)
    }
    stem_course_records.append(record)

print(f"\\n✅ Processed all {len(stem_course_records)} STEM course enrollments")

stem_course_df = pd.DataFrame(stem_course_records)

# Fill NaN values in prerequisite features with appropriate defaults
stem_course_df['avg_grade_in_prerequisites'].fillna(stem_course_df['Student_Mean_GPA'], inplace=True)
stem_course_df['min_grade_in_prerequisites'].fillna(stem_course_df['Student_Mean_GPA'], inplace=True)

print(f"\\nPrerequisite feature statistics:")
print(f"  Students who met all prerequisites: {stem_course_df['all_prerequisites_met'].sum()} ({stem_course_df['all_prerequisites_met'].mean():.1%})")
print(f"  Average grade in prerequisites (mean): {stem_course_df['avg_grade_in_prerequisites'].mean():.2f}")
print(f"  Students with low grades in prerequisites: {(stem_course_df['num_prerequisites_with_low_grades'] > 0).sum()}")

# One-hot encode Student_Program for modeling
stem_course_df_encoded = pd.get_dummies(stem_course_df, columns=['Student_Program'], prefix='Program')

# Get list of program columns created
program_cols = [col for col in stem_course_df_encoded.columns if col.startswith('Program_')]

stem_features = [
    'Student_Mean_GPA', 'Student_CGPA',
    'Course_Difficulty', 'Course_Avg_Grade',
    'Num_Prior_Dept_Courses', 'Avg_Prior_Dept_Grade',
    'Avg_Courses_Per_Semester', 'Std_Courses_Per_Semester', 'Max_Courses_Per_Semester',
    'Program_Avg_Difficulty',
    # NEW: Prerequisite features
    'all_prerequisites_met', 'avg_grade_in_prerequisites', 
    'min_grade_in_prerequisites', 'num_prerequisites_with_low_grades',
    'Is_Local', 'Is_Male', 'Is_STEM_Major',
    'Has_Any_Scholarship',
    'Admission_Year'
] + program_cols  # Add all one-hot encoded program columns

X_stem = stem_course_df_encoded[stem_features]
y_stem = stem_course_df_encoded['Passed']

stem_course_dataset = {
    'X': X_stem,
    'y': y_stem,
    'student_ids': stem_course_df_encoded['Student_Ref'],
    'course_codes': stem_course_df_encoded['Course_Code']
}

print(f"\\nSTEM Course Success Predictor")
print(f"  Dataset size: {len(X_stem)} course enrollments")
print(f"  From {stem_course_df['Student_Ref'].nunique()} unique students")
print(f"  Across {stem_course_df['Course_Code'].nunique()} unique STEM courses")
print(f"  Features: {len(stem_features)}")
print(f"    - Program one-hot encoded: {len(program_cols)} programs")
print(f"  Target distribution:")
print(f"    Passed (Grade ≥ 2.0): {y_stem.sum()} ({y_stem.mean():.1%})")
print(f"    Failed (Grade < 2.0): {(y_stem == 0).sum()} ({(y_stem == 0).mean():.1%})")

STEM course enrollments: 21338
Unique STEM courses: 69
Students who took STEM courses: 3640
\nAdded semester_number to STEM courses dataset
Courses with semester info: 22997
\nCalculating features for each STEM course enrollment (including prerequisites)...
This may take a few minutes...
  Processed 0/22997 enrollments...
  Processed 5000/22997 enrollments...
  Processed 10000/22997 enrollments...
  Processed 15000/22997 enrollments...
  Processed 20000/22997 enrollments...
\n✅ Processed all 22997 STEM course enrollments
\nPrerequisite feature statistics:
  Students who met all prerequisites: 21621 (94.0%)
  Average grade in prerequisites (mean): 2.86
  Students with low grades in prerequisites: 4675
\nSTEM Course Success Predictor
  Dataset size: 22997 course enrollments
  From 3640 unique students
  Across 69 unique STEM courses
  Features: 30
    - Base features: 19 (includes NEW courseload + prerequisite features)
    - Program one-hot encoded: 11 programs
    - Includes Program_Av

### 11.5 Next Semester GPA Change (Regression)

In [94]:
next_sem_data_list = []

for idx, row in master_df.iterrows():
    student_id = row['Student_Ref']
    available_semesters = [col for col in semester_gpa_cols if pd.notna(row[col])]
    
    if len(available_semesters) < 2:
        continue
    
    for i in range(len(available_semesters) - 1):
        current_sem_col = available_semesters[i]
        next_sem_col = available_semesters[i + 1]
        
        current_sem_num = int(current_sem_col.split('_')[1])
        
        current_gpa = row[current_sem_col]
        next_gpa = row[next_sem_col]
        gpa_change = next_gpa - current_gpa
        
        prev_gpas = [row[available_semesters[j]] for j in range(max(0, i-2), i)]
        recent_trend = np.mean(np.diff(prev_gpas)) if len(prev_gpas) > 1 else 0
        recent_volatility = np.std(prev_gpas) if len(prev_gpas) > 1 else 0
        
        # NEW: Get actual courseload for current and prior semesters
        student_courseload_data = semester_courseload[semester_courseload['Student_Ref'] == student_id]
        
        # Current semester courseload
        current_courseload = student_courseload_data[student_courseload_data['semester_number'] == current_sem_num]
        if len(current_courseload) > 0:
            current_courses = current_courseload.iloc[0]['courses_in_semester']
            current_credits = current_courseload.iloc[0]['credits_in_semester']
        else:
            current_courses = row.get('Avg_Courses_Per_Semester', 5)
            current_credits = row.get('Avg_Credits_Per_Semester', 4.5)
        
        # Prior semesters courseload statistics
        prior_courseloads = student_courseload_data[student_courseload_data['semester_number'] < current_sem_num]
        if len(prior_courseloads) > 0:
            avg_prior_courses = prior_courseloads['courses_in_semester'].mean()
            std_prior_courses = prior_courseloads['courses_in_semester'].std() if len(prior_courseloads) > 1 else 0
        else:
            avg_prior_courses = current_courses
            std_prior_courses = 0
        
        record = {
            'Student_Ref': student_id,
            'semester_number': current_sem_num,
            'current_gpa': current_gpa,
            'recent_trend': recent_trend,
            'recent_volatility': recent_volatility,
            'num_semesters_completed': i + 1,
            'current_courses': current_courses,  # NEW: Actual courses this semester
            'current_credits': current_credits,  # NEW: Actual credits this semester
            'avg_prior_courses': avg_prior_courses,  # NEW: Avg courseload before
            'std_prior_courses': std_prior_courses,  # NEW: Courseload variability
            'Is_Local': row['Is_Local'],
            'Is_Male': row['Is_Male'],
            'Is_STEM': row['Is_STEM'],
            'Has_Any_Scholarship': row['Has_Any_Scholarship'],
            'Admission_Year': row['Admission_Year'],
            'gpa_change': gpa_change
        }
        next_sem_data_list.append(record)

next_sem_df = pd.DataFrame(next_sem_data_list)
prog_diff_df = master_df_with_prog_diff[master_df_with_prog_diff['Program_Avg_Difficulty'].notna()].copy()
next_sem_df = next_sem_df.merge(prog_diff_df, on='Student_Ref', how='left', suffixes=('', '_drop'))
# Find all columns that end with '_drop'
cols_to_drop = next_sem_df.filter(regex='_drop$').columns
# Drop these columns
next_sem_df = next_sem_df.drop(columns=cols_to_drop)

next_sem_features = [
    'current_gpa', 'recent_trend', 'recent_volatility', 'num_semesters_completed',
    'current_courses', 'current_credits',  # NEW
    'avg_prior_courses', 'std_prior_courses',  # NEW
    'Is_Local', 'Is_Male', 'Is_STEM', 'Has_Any_Scholarship',
    'Admission_Year',
    'Program_Avg_Difficulty'# maybe add prgm difficulty?
]

X_next_sem = next_sem_df[next_sem_features]
y_next_sem = next_sem_df['gpa_change']

next_semester_dataset = {
    'X': X_next_sem,
    'y': y_next_sem,
    'student_ids': next_sem_df['Student_Ref'],
    'semester_numbers': next_sem_df['semester_number']
}

print("Next Semester GPA Change Prediction (Regression)")
print(f"  Dataset size: {len(X_next_sem)} semester transitions")
print(f"  From {next_sem_df['Student_Ref'].nunique()} unique students")
print(f"  Features: {len(next_sem_features)}")
print(f"    - Includes NEW actual courseload features (semester-specific)")
print(f"  Target statistics (GPA change):")
print(f"    Mean: {y_next_sem.mean():.4f}")
print(f"    Std: {y_next_sem.std():.4f}")
print(f"    Min: {y_next_sem.min():.4f}")
print(f"    Max: {y_next_sem.max():.4f}")
print(f"    % Positive (improving): {(y_next_sem > 0).mean():.1%}")

Next Semester GPA Change Prediction (Regression)
  Dataset size: 20930 semester transitions
  From 3618 unique students
  Features: 14
    - Includes NEW actual courseload features (semester-specific)
  Target statistics (GPA change):
    Mean: 0.0136
    Std: 0.6965
    Min: -4.0000
    Max: 4.0000
    % Positive (improving): 48.8%


### 11.6 Academic Recovery Prediction

**Objective**: For students who go on probation (CGPA < 2.0), predict if they will recover (CGPA ≥ 2.0) in the NEXT semester.

**Key Approach**: Track ALL probation instances (including relapses). Students can recover, fall back into probation, and recover again. This is CRITICAL because 2 consecutive probation semesters → dismissal.

**Features**: Performance at probation, trajectory leading to probation, prior probation history, student characteristics

In [95]:
# Create academic recovery dataset
# Track ALL probation instances, including relapses
# Students can recover from probation, fall back later, and need intervention again

cgpa_copy_sorted = cgpa_copy.sort_values(['Student Ref', 'Academic_Year_Numeric', 'Semester_Numeric'])

recovery_records = []

for student_id, group in cgpa_copy_sorted.groupby('Student Ref'):
    cgpas = group['CGPA'].values
    gpas = group['GPA'].values
    sem_numbers = group['semester_number'].values
    
    # Check ALL semesters for probation (not just first)
    for i, cgpa in enumerate(cgpas):
        # Found a probation semester (CGPA < 2.0)
        if pd.notna(cgpa) and cgpa < 2.0:
            # Check if next semester exists
            if i >= len(cgpas) - 1:
                continue  # No next semester, skip this instance
            
            next_cgpa = cgpas[i + 1]
            if pd.isna(next_cgpa):
                continue  # Next semester has no CGPA data
            
            # Get student-level features
            student_data = master_df[master_df['Student_Ref'] == student_id]
            if len(student_data) == 0:
                continue
            student_data = student_data.iloc[0]
            
            # Calculate features at time of THIS probation instance
            probation_cgpa = cgpa
            probation_gpa = gpas[i]
            sem_at_probation = sem_numbers[i]
            
            # Get prior semester GPAs before THIS probation
            if i > 0:
                prior_gpas = [gpas[j] for j in range(i) if pd.notna(gpas[j])]
                if len(prior_gpas) > 0:
                    mean_prior_gpa = np.mean(prior_gpas)
                    trend_to_probation = np.mean(np.diff(prior_gpas)) if len(prior_gpas) > 1 else 0
                    decline_magnitude = prior_gpas[-1] - probation_gpa  # How much did they drop?
                else:
                    mean_prior_gpa = probation_gpa
                    trend_to_probation = 0
                    decline_magnitude = 0
            else:
                mean_prior_gpa = probation_gpa
                trend_to_probation = 0
                decline_magnitude = 0
            
            # Prior probation history - how many times before THIS instance?
            num_prior_probations = sum(1 for j in range(i) if pd.notna(cgpas[j]) and cgpas[j] < 2.0)
            is_first_probation = int(num_prior_probations == 0)
            
            # Has student recovered from probation before?
            recovered_before = 0
            if num_prior_probations > 0:
                # Check if any probation before this one led to recovery
                for j in range(i):
                    if pd.notna(cgpas[j]) and cgpas[j] < 2.0 and j < len(cgpas) - 1:
                        if pd.notna(cgpas[j + 1]) and cgpas[j + 1] >= 2.0:
                            recovered_before = 1
                            break
            
            # NEW: Get courseload statistics up to probation semester
            student_courseload_data = semester_courseload[semester_courseload['Student_Ref'] == student_id]
            prior_courseloads = student_courseload_data[student_courseload_data['semester_number'] <= sem_at_probation]
            
            if len(prior_courseloads) > 0:
                avg_courses_per_sem = prior_courseloads['courses_in_semester'].mean()
                std_courses_per_sem = prior_courseloads['courses_in_semester'].std() if len(prior_courseloads) > 1 else 0
                max_courses_per_sem = prior_courseloads['courses_in_semester'].max()
            else:
                avg_courses_per_sem = student_data.get('Avg_Courses_Per_Semester', 5)
                std_courses_per_sem = student_data.get('Std_Courses_Per_Semester', 0)
                max_courses_per_sem = avg_courses_per_sem
            
            # Target: Did student recover (CGPA ≥ 2.0) in next semester?
            recovered = int(next_cgpa >= 2.0)
            
            record = {
                'Student_Ref': student_id,
                'probation_instance': num_prior_probations + 1,  # Which probation episode is this?
                'is_first_probation': is_first_probation,
                'probation_cgpa': probation_cgpa,
                'probation_gpa': probation_gpa,
                'semester_of_probation': sem_at_probation,
                'mean_gpa_before_probation': mean_prior_gpa,
                'trend_to_probation': trend_to_probation,
                'decline_magnitude': decline_magnitude,
                'num_prior_probations': num_prior_probations,
                'recovered_from_probation_before': recovered_before,
                'semesters_before_probation': i,
                'avg_courses_per_sem': avg_courses_per_sem,  # NEW: Actual courseload
                'std_courses_per_sem': std_courses_per_sem,  # NEW: Courseload variability
                'max_courses_per_sem': max_courses_per_sem,  # NEW: Peak courseload
                'Is_Local': student_data['Is_Local'],
                'Is_Male': student_data['Is_Male'],
                'Is_STEM': student_data['Is_STEM'],
                'Has_Need_Based_Scholarship': student_data['Has_Need_Based_Scholarship'],
                'Has_Any_Scholarship': student_data['Has_Any_Scholarship'],
                'Admission_Year': student_data['Admission_Year'],
                'recovered': recovered
            }
            recovery_records.append(record)

recovery_df_model = pd.DataFrame(recovery_records)
prog_diff_df = master_df_with_prog_diff[master_df_with_prog_diff['Program_Avg_Difficulty'].notna()].copy()
recovery_df_model = recovery_df_model.merge(prog_diff_df, on='Student_Ref', how='left', suffixes=('', '_drop'))
# Find all columns that end with '_drop'
cols_to_drop = recovery_df_model.filter(regex='_drop$').columns
# Drop these columns
recovery_df_model = recovery_df_model.drop(columns=cols_to_drop)


recovery_features = [
    'is_first_probation', 'probation_cgpa', 'probation_gpa', 'semester_of_probation',
    'mean_gpa_before_probation', 'trend_to_probation', 'decline_magnitude',
    'num_prior_probations', 'recovered_from_probation_before',
    'semesters_before_probation',
    'avg_courses_per_sem', 'std_courses_per_sem', 'max_courses_per_sem',  # NEW
    'Is_Local', 'Is_Male', 'Is_STEM',
    'Has_Need_Based_Scholarship', 'Has_Any_Scholarship',
    'Admission_Year',
    'Program_Avg_Difficulty'# maybe add program difficulty?
]

X_recovery = recovery_df_model[recovery_features]
y_recovery = recovery_df_model['recovered']

academic_recovery_dataset = {
    'X': X_recovery,
    'y': y_recovery,
    'student_ids': recovery_df_model['Student_Ref'],
    'probation_instance': recovery_df_model['probation_instance']
}

print("Academic Recovery Prediction (ALL PROBATION INSTANCES)")
print(f"  Dataset size: {len(X_recovery)} probation-to-next-semester cases")
print(f"  Unique students: {recovery_df_model['Student_Ref'].nunique()}")
print(f"  Features: {len(recovery_features)}")
print(f"    - Includes NEW actual courseload statistics")

# Analyze relapse patterns
students_with_multiple = recovery_df_model['Student_Ref'].value_counts()
students_with_relapses = (students_with_multiple > 1).sum()

print(f"\n  Relapse tracking:")
print(f"    Students with single probation: {(students_with_multiple == 1).sum()}")
print(f"    Students with multiple probations (relapses): {students_with_relapses}")
print(f"    Average probation episodes per student: {students_with_multiple.mean():.2f}")

print(f"\n  Target distribution:")
print(f"    Recovered (CGPA ≥ 2.0 next semester): {y_recovery.sum()} ({y_recovery.mean():.1%})")
print(f"    Did not recover: {(y_recovery == 0).sum()} ({(y_recovery == 0).mean():.1%})")

print(f"\n  First vs subsequent probations:")
print(f"    First probation instances: {recovery_df_model['is_first_probation'].sum()}")
print(f"    Subsequent probation instances (relapses): {(recovery_df_model['is_first_probation'] == 0).sum()}")

print(f"\n  TRACKS ALL INSTANCES: Including relapses ({students_with_relapses} students)")
print(f"  CRITICAL: Predicts dismissal risk (2 consecutive probations → dismissal)")
print(f"  ACTIONABLE: Target interventions after ANY probation episode")

Academic Recovery Prediction (ALL PROBATION INSTANCES)
  Dataset size: 937 probation-to-next-semester cases
  Unique students: 365
  Features: 20
    - Includes NEW actual courseload statistics

  Relapse tracking:
    Students with single probation: 164
    Students with multiple probations (relapses): 201
    Average probation episodes per student: 2.57

  Target distribution:
    Recovered (CGPA ≥ 2.0 next semester): 329 (35.1%)
    Did not recover: 608 (64.9%)

  First vs subsequent probations:
    First probation instances: 365
    Subsequent probation instances (relapses): 572

  TRACKS ALL INSTANCES: Including relapses (201 students)
  CRITICAL: Predicts dismissal risk (2 consecutive probations → dismissal)
  ACTIONABLE: Target interventions after ANY probation episode


## 12. Save All Prepared Datasets

In [96]:
import pickle

datasets = {
    'early_warning_1st_semester': early_warning_1st_sem_data,
    'early_warning_1st_year': early_warning_1st_year_data,
    'semester_probation_prediction': probation_dataset,
    'academic_recovery': academic_recovery_dataset,
    'next_semester_gpa_change': next_semester_dataset,
    'stem_course_success': stem_course_dataset
}

with open('supervised_learning_datasets.pkl', 'wb') as f:
    pickle.dump(datasets, f)

print("✅ All datasets saved to 'supervised_learning_datasets.pkl'")
print("\n" + "="*70)
print("DATASET SUMMARY - 6 MODELS READY FOR TRAINING")
print("="*70)

for name, data in datasets.items():
    print(f"\n{name.replace('_', ' ').title()}:")
    print(f"  Samples: {len(data['X']):,}")
    print(f"  Features: {len(data['X'].columns)}")
    if 'y_multiclass' in data:
        print(f"  Type: Multi-class Classification (Grade Bin)")
        print(f"  Classes: {data['y_multiclass'].nunique()}")
    elif data['y'].dtype in ['float64', 'float32']:
        print(f"  Type: Regression")
        print(f"  Target mean: {data['y'].mean():.3f}")
    else:
        print(f"  Type: Binary Classification")
        print(f"  Positive class: {data['y'].mean():.1%}")


✅ All datasets saved to 'supervised_learning_datasets.pkl'

DATASET SUMMARY - 6 MODELS READY FOR TRAINING

Early Warning 1St Semester:
  Samples: 3,718
  Features: 25
  Type: Multi-class Classification (Grade Bin)
  Classes: 4

Early Warning 1St Year:
  Samples: 3,618
  Features: 30
  Type: Multi-class Classification (Grade Bin)
  Classes: 4

Semester Probation Prediction:
  Samples: 20,930
  Features: 21
  Type: Binary Classification
  Positive class: 3.9%

Academic Recovery:
  Samples: 937
  Features: 20
  Type: Binary Classification
  Positive class: 35.1%

Next Semester Gpa Change:
  Samples: 20,930
  Features: 14
  Type: Regression
  Target mean: 0.014

Stem Course Success:
  Samples: 22,997
  Features: 30
  Type: Binary Classification
  Positive class: 82.1%
